# 1. MultiThread for Crawl User_info from list of UserURLs

In [1]:
from facebook_scraper import get_profile
get_profile("profile.php?id=100071755697362") # Or get_profile("zuck", cookies="cookies.txt")

c:\Users\LENOVO\miniconda3\envs\fb_crawl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'Friend_count': None,
 'Follower_count': None,
 'Following_count': None,
 'Name': 'Minh Huyềnn'}

In [2]:

from facebook_scraper import get_profile
get_profile("hongtieungan") # Or get_profile("zuck", cookies="cookies.txt")

{'Friend_count': None,
 'Follower_count': None,
 'Following_count': None,
 'Name': 'Tiểuu Ngânn'}

In [12]:

from facebook_scraper import get_profile
get_profile("https://www.facebook.com/profile.php?id=100071755697362") # Or get_profile("zuck", cookies="cookies.txt")

HTTPError: 404 Client Error: Not Found for url: https://m.facebook.com/https://www.facebook.com/100071755697362/about/

In [1]:
%cd ../src

d:\UNI\facebook_crawler\src


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_argument("--incognito")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_user_info(driver, user_list, start_index, chunk_size, result):
    user_crawl =  user_profile.UserInfo(driver)
    for i in range(start_index, start_index+chunk_size):
        user_info = user_crawl.get_user_info(user_list[i])
        result[i] = user_info


username = "rinputin482@gmail.com"
password = 'Rinputin482qh'

no_thread = 3
base_url = 'https://www.facebook.com/'
drivers = [initialize_driver() for _ in range(no_thread)]

for driver in drivers:
    driver.get(base_url)
    login(driver, username, password)
    time.sleep(3)



def crawl_info_multithread(drivers, user_list, no_thread):
    result = [None] * len(user_list)
    chunk_size = len(user_list) // no_thread
    start = 0
    threads = []
    for driver in drivers:
        thread = threading.Thread(target=crawl_user_info, args=(driver, user_list, start, chunk_size, result))
        threads.append(thread)
        start += chunk_size
    start_time = time.perf_counter()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(time.perf_counter() - start_time, 'seconds')
    return result

23:45:19 utils.Login: Login Successfull
23:45:19 utils.Login: Login Successfull
23:45:19 utils.Login: Login Successfull


In [11]:
columns = ['url','name', 'Gender', 'Birthdate', 'Birthyear', 'Highschool', 'College', 'Currentcity', 'Hometown', 'group']

def merge_dict(raw_data_dict, group, url):
    name, basic_inf, school, locate =  raw_data_dict
    raw_data = {'url':url, 'name':name, 'group': group , **basic_inf, **school, **locate}
    data = {}
    keys = raw_data.keys()
    for i in columns:
        if i in keys:
            i = i.replace(' ','')
            if i=='Highschool' or i=='College':
                data[i] = raw_data[i].replace('Studied at ', '').replace('Went to ', '')
            else:
                data[i] = raw_data[i]
        else:
            data[i] = None
    return data

In [12]:
def generate_batches(start, end, batch_size):
    batches = []
    current_start = start

    while current_start < end:
        current_end = min(current_start + batch_size, end)
        batches.append((current_start, current_end))
        current_start = current_end

    return batches

start = 0
end = 80
batch_size = 30

batch_list = generate_batches(start, end, batch_size)
batch_list

[(0, 30), (30, 60), (60, 80)]

In [8]:
import pandas as pd
start = 6001
df = pd.read_csv('../datasets/fake_taxi.csv')
end = 9000
no_to_crawl = end - start
no_thread = 3
batch_size = 60
save_path = '../datasets/user_info_from_friend.csv'



batch_list = generate_batches(start, end, batch_size)
batch_list

[(6001, 6061),
 (6061, 6121),
 (6121, 6181),
 (6181, 6241),
 (6241, 6301),
 (6301, 6361),
 (6361, 6421),
 (6421, 6481),
 (6481, 6541),
 (6541, 6601),
 (6601, 6661),
 (6661, 6721),
 (6721, 6781),
 (6781, 6841),
 (6841, 6901),
 (6901, 6961),
 (6961, 7021),
 (7021, 7081),
 (7081, 7141),
 (7141, 7201),
 (7201, 7261),
 (7261, 7321),
 (7321, 7381),
 (7381, 7441),
 (7441, 7501),
 (7501, 7561),
 (7561, 7621),
 (7621, 7681),
 (7681, 7741),
 (7741, 7801),
 (7801, 7861),
 (7861, 7921),
 (7921, 7981),
 (7981, 8041),
 (8041, 8101),
 (8101, 8161),
 (8161, 8221),
 (8221, 8281),
 (8281, 8341),
 (8341, 8401),
 (8401, 8461),
 (8461, 8521),
 (8521, 8581),
 (8581, 8641),
 (8641, 8701),
 (8701, 8761),
 (8761, 8821),
 (8821, 8881),
 (8881, 8941),
 (8941, 9000)]

In [14]:
import pandas as pd
start = 0
df = pd.read_csv('../datasets/fake_taxi.csv')
end = 3000
no_to_crawl = end - start
no_thread = 3
batch_size = 60
save_path = '../datasets/user_info_from_friend.csv'
df
batch_list = generate_batches(start, end, batch_size)
batch_list

[(0, 60),
 (60, 120),
 (120, 180),
 (180, 240),
 (240, 300),
 (300, 360),
 (360, 420),
 (420, 480),
 (480, 540),
 (540, 600),
 (600, 660),
 (660, 720),
 (720, 780),
 (780, 840),
 (840, 900),
 (900, 960),
 (960, 1020),
 (1020, 1080),
 (1080, 1140),
 (1140, 1200),
 (1200, 1260),
 (1260, 1320),
 (1320, 1380),
 (1380, 1440),
 (1440, 1500),
 (1500, 1560),
 (1560, 1620),
 (1620, 1680),
 (1680, 1740),
 (1740, 1800),
 (1800, 1860),
 (1860, 1920),
 (1920, 1980),
 (1980, 2040),
 (2040, 2100),
 (2100, 2160),
 (2160, 2220),
 (2220, 2280),
 (2280, 2340),
 (2340, 2400),
 (2400, 2460),
 (2460, 2520),
 (2520, 2580),
 (2580, 2640),
 (2640, 2700),
 (2700, 2760),
 (2760, 2820),
 (2820, 2880),
 (2880, 2940),
 (2940, 3000)]

In [16]:
from tqdm import tqdm
from utils.action import surf_feed
flag = time.time()
for i in tqdm(batch_list, desc="Processing batches"):
    start,end = i
    print(start,end)
    chunk_df = df[start:end]
    chunk_df.reset_index(drop=True, inplace=True)
    user_list = chunk_df['url'].to_list()
    result = crawl_info_multithread(drivers, user_list, no_thread)

    data = []
    for i in range(len(user_list)):
        group = chunk_df['from'][i]
        url = chunk_df['url'][i]
        try:
            new_dict = merge_dict(result[i], group, url)
            data.append(new_dict)
        except:
            continue
    crawl_df = pd.DataFrame(data)
    existing_df = pd.read_csv(save_path)
    existing_df = pd.concat([existing_df, crawl_df], ignore_index=True)
    existing_df.to_csv(save_path, index=False)
    time.sleep(3)

    if (time.time() - flag) > 60 * 60:
        drivers[1].get(base_url)
        drivers[2].get(base_url)
        surf_feed(drivers[0])
        flag = time.time()


Processing batches:   0%|          | 0/50 [00:00<?, ?it/s]

0 60


Exception in thread Thread-28:
Traceback (most recent call last):
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_5220\4215338391.py", line 23, in crawl_user_info
  File "d:\UNI\facebook_crawler\src\user_profile.py", line 22, in get_user_info
    name, basic_info_dict = self._get_basic_info()
  File "d:\UNI\facebook_crawler\src\user_profile.py", line 35, in _get_basic_info
    name = get_element.get_element(browser=self.browser, xpath=xPath_name).text
AttributeError: 'NoneType' object has no attribute 'text'
Exception in thread Thread-30:
Traceback (most recent call last):
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\threading.py", line 954, in _bootstrap_inner
   

671.1416543580001 seconds


Processing batches:   2%|▏         | 1/50 [12:25<10:09:11, 745.95s/it]

...

1020 1080
0.04400465099979556 seconds


Processing batches:  36%|███▌      | 18/50 [13:51<01:55,  3.61s/it]Exception in thread Exception in thread Thread-83:
Traceback (most recent call last):
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\threading.py", line 954, in _bootstrap_inner
Thread-82:
Traceback (most recent call last):
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_5220\4215338391.py", line 23, in crawl_user_info
    self.run()
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_5220\4215338391.py", line 23, in crawl_user_info
Exception i

1080 1140
0.047576416000083555 seconds


Processing batches:  36%|███▌      | 18/50 [13:55<24:44, 46.39s/it]


KeyboardInterrupt: 

In [7]:
import pandas as pd
start = 0
df = pd.read_csv('../datasets/data.csv')
end = 10000
no_to_crawl = end - start
no_thread = 3
batch_size = 30
save_path = '../datasets/user_info_from_group.csv'



batch_list = generate_batches(start, end, batch_size)
from tqdm import tqdm
for i in tqdm(batch_list, desc="Processing batches"):
    start,end = i
    print(start,end)
    chunk_df = df[start:end]
    chunk_df.reset_index(drop=True, inplace=True)
    user_list = chunk_df['member_url'].to_list()
    result = crawl_info_multithread(drivers, user_list, no_thread)

    data = []
    for i in range(len(user_list)):
        group = chunk_df['group'][i]
        url = chunk_df['member_url'][i]
        try:
            new_dict = merge_dict(result[i], group, url)
            data.append(new_dict)
        except:
            continue
    crawl_df = pd.DataFrame(data)
    existing_df = pd.read_csv(save_path)
    existing_df = pd.concat([existing_df, crawl_df], ignore_index=True)
    existing_df.to_csv(save_path, index=False)
    time.sleep(3)


Processing batches:   0%|          | 0/334 [00:00<?, ?it/s]

0 30
353.7120695 seconds


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19724\3391499939.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  existing_df = pd.concat([existing_df, crawl_df], ignore_index=True)
Processing batches:   0%|          | 1/334 [05:56<33:01:15, 356.98s/it]

30 60
331.9490574 seconds


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19724\3391499939.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  existing_df = pd.concat([existing_df, crawl_df], ignore_index=True)
Processing batches:   1%|          | 2/334 [11:31<31:43:45, 344.05s/it]

...

Processing batches: 100%|█████████▉| 333/334 [2:31:01<00:08,  8.37s/it]

9990 10000


Exception in thread Thread-1013:
Traceback (most recent call last):
  File "c:\Users\LENOVO\miniconda3\envs\fb_crawl\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\LENOVO\miniconda3\envs\fb_crawl\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\LENOVO\miniconda3\envs\fb_crawl\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19724\3533052625.py", line 23, in crawl_user_info
  File "c:\Users\LENOVO\Desktop\facebook_crawler\src\user_profile.py", line 22, in get_user_info
    name, basic_info_dict = self._get_basic_info()
  File "c:\Users\LENOVO\Desktop\facebook_crawler\src\user_profile.py", line 36, in _get_basic_info
    name = get_element.get_element(browser=self.browser, xpath=xPath_name).text
AttributeError: 'NoneType' object has no attribute 'text'
Exception in thread Thread-1011:
Traceback (most rec

10.882745299999442 seconds


Processing batches: 100%|██████████| 334/334 [2:31:15<00:00, 27.17s/it]


# 2. MultiThread for Crawl multiple group members from list of groups

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile
import group

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_group_member(driver, group_list, start_index, chunk_size, result):
    group_crawler = group.Group(browser=driver)
    for i in range(start_index, start_index+chunk_size):
        try:
            user_list = group_crawler.get_url_from_members(group_list[i], max_user=100)
        except:
            user_list = None
        result[i] = user_list


username = "lth2112002@gmail.com"
password = 'Baoanh123'


group_links=[
    'https://www.facebook.com/groups/932212904285349',
    'https://www.facebook.com/groups/1986195691670003',
    'https://www.facebook.com/groups/263503162332380',
    'https://www.facebook.com/groups/flexdenhoithocuoicung',
    'https://www.facebook.com/groups/nthehatoansongsong',
]

base_url = 'https://www.facebook.com/'
drivers = [initialize_driver() for _ in range(5)]

for driver in drivers:
    driver.get(base_url)
    login(driver, username, password)

result = [None] * len(group_links)

chunk_size = len(group_links) // 5   # Max = 5
start = 0

threads = []
for driver in drivers:
    thread = threading.Thread(target=crawl_group_member, args=(driver, group_links, start, chunk_size, result))
    threads.append(thread)
    start += chunk_size

start_time = time.perf_counter()

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

print(time.perf_counter() - start_time, 'seconds')

for driver in drivers:
    driver.quit()

Success Login
Success Login
Success Login
Success Login
Success Login
Start Crawling User from Group: CLB Sáng tạo trẻ - Trường THPT A Lưới
===73/100
===93/100
===103/100


100%|██████████| 100/100 [00:01<00:00, 62.72it/s]


20.2755345999999 seconds


In [7]:
result

[None,
 ['https://www.facebook.com/profile.php?id=100071684331377',
  'https://www.facebook.com/profile.php?id=100015733534289',
  'https://www.facebook.com/profile.php?id=100013886446373',
  'https://www.facebook.com/profile.php?id=61555335785751',
  'https://www.facebook.com/profile.php?id=100092380986251',
  'https://www.facebook.com/profile.php?id=100082605951622',
  'https://www.facebook.com/profile.php?id=100003046968242',
  'https://www.facebook.com/profile.php?id=100003677062318',
  'https://www.facebook.com/profile.php?id=100008105701061',
  'https://www.facebook.com/profile.php?id=100066320130406',
  'https://www.facebook.com/profile.php?id=100077485805711',
  'https://www.facebook.com/profile.php?id=61551622838976',
  'https://www.facebook.com/profile.php?id=100076634932874',
  'https://www.facebook.com/profile.php?id=100084516473525',
  'https://www.facebook.com/profile.php?id=100084144864498',
  'https://www.facebook.com/profile.php?id=100031024537497',
  'https://www.face

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile
import group

username = "lth2112002@gmail.com"
password = 'Baoanh123'


group_links=[
'https://www.facebook.com/groups/932212904285349',
'https://www.facebook.com/groups/1986195691670003',
    'https://www.facebook.com/groups/263503162332380',
    'https://www.facebook.com/groups/flexdenhoithocuoicung',
    'https://www.facebook.com/groups/nthehatoansongsong',
]

base_url = 'https://www.facebook.com/'

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_group_member(driver, group_list, start_index, chunk_size, result):
    group_crawler = group.Group(browser=driver)
    for i in range(start_index, start_index+chunk_size):
        try:
            user_list = group_crawler.get_url_from_members(group_list[i], max_user=100)
        except:
            user_list = None
        result[i] = user_list

def multi_thread_crawl(group_links,num_thread=5):

    drivers = [initialize_driver() for _ in range(5)]

    for driver in drivers:
        driver.get(base_url)
        login(driver, username, password)

    result = [None] * len(group_links)
    chunk_size = len(group_links) //  num_thread  # Max = 5
    start = 0
    threads = []
    for driver in drivers:
        thread = threading.Thread(target=crawl_group_member, args=(driver, group_links, start, chunk_size, result))
        threads.append(thread)
        start += chunk_size

    start_time = time.perf_counter()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(time.perf_counter() - start_time, 'seconds')

    for driver in drivers:
        driver.quit()
    
    return result